# Generation with LLMs

LLMs are the key component behind text generation.

Autoregressive generation is the inference-time procedure of iteratively calling a model with its own generated outputs, given a few initial inputs. In Transformers library, this is handled by the `generate()` method, which is available to all models with generative capabilities.

In [1]:
!pip install transformers bitsandbytes>=0.39.0 -q

## Generate text

A language model trained for **causal language modeling** takes a sequence of text tokens as input and returns the probability distribution for the next token.

A critical aspect of autoregressive generation with LLMs is how to select the next token from this probability distribution. Anything goes in this step as long as we end up with a token for the next iteration. This means it can be as simple as selecting the most likely token from the probability distribution or as complex as applying a dozen transformations before sampling from the resulting distribution.

Ideally, the stopping condition is dictated by the model, which should learn when to output an end-of-sequence (`EOS`) token. If this is not the case, generation stops when some predefined maximum length is reached.

Properly setting up the token selection step and the stopping condition is essential to make our model behave as we would expect on our task. This is why we have a `GenerationConfig` file associated with each model, which contains a good default generative parameterization and is loaded alongside our model.

First, we need to load the model.

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mistral-7B-v0.1',
    device_map='auto',
    load_in_4bit=True,
)

In the `from_pretrained()` call:
* `device_map` ensures the model is move to GPU(s)
* `load_in_4bit` applies 4-bit dynamic quantization to massively reduce the resource requirements

Next, we need to preprocess our text input with a tokenizer:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1', padding_side='left')

In [ ]:
model_inputs = tokenizer(['A list of colors: red, blue, green'],
                         return_tensors='pt').to('cuda')

The `model_inputs` variable holds the tokenized text input, as well as the attention mask. While `generate()` does its best effort to infer the attention mask when it is not passed, we still recommend passing it whenever possible for optimal results.

After tokenizing the inputs, we can call the `generate()` method to return the generated results. The generated tokens should be converted to text before printing:

In [ ]:
generated_ids = model.generate(**model_inputs)

tokenizer.batch_decode(generated_ids, skip_speical_tokens=True)[0]

Finally, we don't need to do it one sequence at a time. We can batch our inputs, which will greatly improve the throughput at a small lantency and memory cost. All we need to do is to make sure we pad our inputs properly:

In [ ]:
tokenizer.pad_token = tokenizer.eos_token # most LLMs don't have a pad token by default

model_inputs = tokenizer(
    ['A list of colors: red, blue, green', 'Paris is'],
    return_tensors='pt',
    padding=True,
).to('cuda')

generated_ids = model.generate(**model_inputs)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

## Common pitfalls

There are many generation strategies, and sometimes the default values may not be appropriate for our use case. If our outputs are not aligned with what we are expecting, we have created a list of the most common pitfalls and how to avoid them.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_checkpoint = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token # most LLMs don't have a pad token by default
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    device_map='auto',
    load_in_4bit=True,
)

### Generated output is too short/long

If not specified in he `GenerationConfig` file, `generate` returns up to 20 tokens by default.

LLMs (more precisely, decoder-only models) also return the input prompt as part of the output.

In [ ]:
model_inputs = tokenizer(
    ['A sequence of numbers: 1, 2'],
    return_tensors = 'pt',
).to('cuda')

In [ ]:
# by default, the output will contain up to 20 tokens
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
# setting `max_new_tokens` allows us to control the maximum length
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### Incorrect generation mode

By default, `generate` selects the most likely token at each iteration (greedy decoding). Depending on our task, this may be undesirable; creative tasks like chatbots or writing an essay benefit from sampling.

Enable sampling with `do_sampling=True`:

In [ ]:
# set seed for reproducibility
from transformers import set_seed
set_seed(101)

model_inputs = tokenizer(
    ['I am a cat.'],
    return_tensors = 'pt',
).to('cuda')

In [ ]:
# LLM + greedy decoding
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
# With sampling, the output becomes more creative
generated_ids = model.generate(**model_inputs, do_sample=True)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### Wrong padding side

LLMs are decoder-only architectures, meaning they continue to iterate on our input prompt. Since LLMs are not trained to continue from pad tokens, our input needs to be left-padded. Make sure we also do not forget to pass the attention mask to generate.

In [ ]:
# The tokenizer initialized above has right-padding active by default
model_inputs = tokenizer(
    ['1, 2, 3', 'A, B, C, D, E'],
    padding=True,
    return_tensors='pt',
).to('cuda')

generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
# With left-padding, it works as expected
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token # most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ['1, 2, 3', 'A, B, C, D, E'],
    padding=True,
    return_tensors='pt',
).to('cuda')

generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

### Wrong prompt

Some models and tasks expect a certain input prompt format to work properly.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_checkpoint = 'HuggingFaceH4/zephyr-7b-alpha'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    device_map='auto',
    load_in_4bit=True,
)

In [ ]:
set_seed(101)
prompt = """How many helicopters can a human eat in one sitting? Reply as a thug."""
model_inputs = tokenizer([prompt], return_tensors='pt').to('cuda')
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs, max_new_tokens=20)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

In [ ]:
set_seed(101)
messages = [
    {
        'role': 'system',
        'content': 'You are a friendly chatbot who always responds in the style of a thug',
    },
    {
        'role': 'user',
        'content': 'How many helicopters can a human eat in one sitting?',

    },
]
model_inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors='pt',
).to('cuda')
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=20)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])